In [4]:
from tqdm import tqdm
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from src import constant
from robustbench import load_model
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.9.0
Torchvision Version:  0.10.0


In [13]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "Standard"
num_classes = 10
batch_size = 8
num_epochs = 15
feature_extract = False

In [25]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [26]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [27]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}
dataset = torchvision.datasets.CIFAR10(root=constant.DATA_DIR, train=True, download=True,
                                           transform=torchvision.transforms.ToTensor())
image_datasets = {
    'train': torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transforms['train'], download=True),
    'val': torchvision.datasets.CIFAR10(root='./data', train=False, transform=data_transforms['val'], download=True)
}
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=False, num_workers=4)
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = load_model("Standard")
set_parameter_requires_grad(model_ft, feature_extract)
model_ft = model_ft.to(device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [28]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 conv1.weight
	 block1.layer.0.bn1.weight
	 block1.layer.0.bn1.bias
	 block1.layer.0.conv1.weight
	 block1.layer.0.bn2.weight
	 block1.layer.0.bn2.bias
	 block1.layer.0.conv2.weight
	 block1.layer.0.convShortcut.weight
	 block1.layer.1.bn1.weight
	 block1.layer.1.bn1.bias
	 block1.layer.1.conv1.weight
	 block1.layer.1.bn2.weight
	 block1.layer.1.bn2.bias
	 block1.layer.1.conv2.weight
	 block1.layer.2.bn1.weight
	 block1.layer.2.bn1.bias
	 block1.layer.2.conv1.weight
	 block1.layer.2.bn2.weight
	 block1.layer.2.bn2.bias
	 block1.layer.2.conv2.weight
	 block1.layer.3.bn1.weight
	 block1.layer.3.bn1.bias
	 block1.layer.3.conv1.weight
	 block1.layer.3.bn2.weight
	 block1.layer.3.bn2.bias
	 block1.layer.3.conv2.weight
	 block2.layer.0.bn1.weight
	 block2.layer.0.bn1.bias
	 block2.layer.0.conv1.weight
	 block2.layer.0.bn2.weight
	 block2.layer.0.bn2.bias
	 block2.layer.0.conv2.weight
	 block2.layer.0.convShortcut.weight
	 block2.layer.1.bn1.weight
	 block2.layer.1.bn1.bias


In [29]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [30]:
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

  0%|          | 0/6250 [00:00<?, ?it/s]

Epoch 0/14
----------


  9%|▉         | 589/6250 [08:42<1:28:14,  1.07it/s]